# Running a MCMC Bayeslands example

In this notebook, we will and run a simple Bayeslands model using a mcmc chain.

## Bayes

Given some data, $D$, we make inference regarding unknown parameters, denoted by  $\theta$, via the posterior distribution $p(\theta|D)$,

given by Bayes rule, $p(\theta|D)=\frac{p(D|\theta)p(\theta)}{P(D)}$

$p(D|\theta)$ is the likelihood of the data given the parameters

$p(\theta)$ is the prior

$p(D)$ is a normalizing constant and equal to $\int p(D|\theta)p(\theta)d\theta$. 

Bayeslands uses MCMC to obtain samples of $\theta$ from the posterior distribution by proposing values from some known distribution $q(\theta)$. The value of $\theta$ is  then set equal to these proposed values with a probability which ensures the detailed balance condition is met, otherwise the chain remains in its current position. The transition kernel which moves the Markov chain from one point to another in the parameter space is the product of the proposal distribution $q$ and the acceptance probability, $\alpha$. Under certain conditions, the draws from this transition kernel converge to draws from the stationary distribution, $p(\theta| D)$.

## Markov Chain Monte Carlo

This method requires running Badlands thousands of times to explore the full parameter space. If a model takes 1 second, this is okay, but if a model takes a few minutes, then it does not scale very well and for that purpose we'll be using the concept of ***parallel tempering*** to help overcome this problem.

The MCMC Metropolis Hastings algorithm can be visualised as follows:

   Metropolis Hastings |  Algorithm Workflow 
:-------------------------:|:-------------------------:
![](notebook_images/alg_MH.png)  |  ![](notebook_images/Bayeslands-MCMC-flowchart.png)

Here are the trace plots for the two parameters of rainfall and erodibility. Each sample is trying a slightly different value of rainfall and erodibility (combined with other parametrs). In this case the ***true value*** of rainfall and erodibility was 1.5 m/yr and 0.00005 respectively.

Parameter exploration with MCMC
![](notebook_images/trace_mcmc_rain_erod.png)





In [ ]:
#Set up the environemnt
%matplotlib inline
import sys
sys.path.append('scripts/')

import numpy as np
import time
import random

#Import the script that produces a start and end topography
from bl_mcmc import *

# from bl_preproc import *
from bl_postproc import *

# display plots in SVG format
#%config InlineBackend.figure_format = 'svg'

In [ ]:
random.seed(time.time())
muted = True
run_nb = 0
directory = ""
likl_sed = False
erdp_coords = np.array([[60,60],[52,67],[74,76],[62,45],[72,66],[85,73],[90,75],[44,86],[100,80],[88,69]])

directory = 'crater'
xmlinput = '%s/crater.xml' %(directory)
samples = 20
simtime = 50000
rainlimits = [0.0, 3.0]
erodlimits = [3.e-5, 7.e-5]
mlimit = [0.4, 0.6]
nlimit = [0.9, 1.1]
true_rain = 1.5
true_erod = 5.e-5
likl_sed = True
erdp_coords = erdp_coords


In [ ]:
#Create all the output directories
final_elev = np.loadtxt('%s/data/final_elev.txt' %(directory))
final_erdp = np.loadtxt('%s/data/final_erdp.txt' %(directory))
final_erdp_pts = np.loadtxt('%s/data/final_erdp_pts.txt' %(directory))	

while os.path.exists('%s/mcmcresults_%s' % (directory,run_nb)):
    run_nb+=1
if not os.path.exists('%s/mcmcresults_%s' % (directory,run_nb)):
    os.makedirs('%s/mcmcresults_%s' % (directory,run_nb))
    os.makedirs('%s/mcmcresults_%s/plots' % (directory,run_nb))
    os.makedirs('%s/mcmcresults_%s/prediction_data' % (directory,run_nb))
    filename = ('%s/mcmcresults_%s' % (directory,run_nb))

print '\nInput file shape', final_elev.shape, '\n'
run_nb_str = 'mcmcresults_' + str(run_nb)


In [ ]:
#Make the Bayeslands Sampler
bl_mcmc = bayeslands_mcmc(muted, simtime, samples, final_elev, final_erdp, final_erdp_pts, erdp_coords, filename, xmlinput, erodlimits, rainlimits, mlimit, nlimit, run_nb_str, likl_sed)


In [ ]:
#And run it
bl_mcmc.sampler()

print '\nsuccessfully sampled\nFinished simulations'

# Check the output

In [ ]:
run_nb = input('Please enter the folder number for Experiment i.e. mcmcresults_% ')
fname = '%s/mcmcresults_%s/' % (directory,run_nb)
exp_data = '%s/mcmcresults_%s/exp_data.txt' % (directory,run_nb)
prediction_data = '%s/mcmcresults_%s/prediction_data/' % (directory,run_nb)

In [ ]:
filename_list = []
filename_list.append(exp_data)

rain = []
erod = []
likl = []

for list_name in filename_list:
    with open(list_name) as f:
        for count, line in enumerate(f):
            # print count,' ', line
            term = line.split()
            # print 'term', term
            rain.append(term[0])
            # print 'rain', rain
            erod.append(term[1])
            # print 'erod', erod
            likl.append(term[2])
            # print 'likl', likl

print 'length of likl', len(likl), ' rain', len(rain), ' erod', len(erod)

rain_ = np.asarray(rain, dtype = float)
erod_ = np.asarray(erod, dtype = float)
likl_ = np.asarray(likl, dtype = float)
t_val_ = np.loadtxt('%s/data/true_values.txt' % (directory))
erdp_pts_data = np.loadtxt('%s/data/final_erdp_pts.txt' % (directory))
prefixed = [filename for filename in os.listdir(prediction_data) if filename.startswith("mean_pred_erdp_pts_")]

bins= input("Please enter Bin Size for Histogram (e.g. 10, 20 ,30 ...")
plotFunctions(fname, likl_, rain_, erod_, bins, t_val_)
# plotFunctions(fname, likl_, rain_, erod_, bins, rain_true_val, erod_true_val)
print '\nFinished plotting'
